Paul Mooney, 11/29/2017

**In this year's Halloween playground competition, you're challenged to predict the author of excerpts from horror stories by Edgar Allan Poe, Mary Shelley, and HP Lovecraft**

In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
import keras
import keras.backend as K
from keras.layers import Dense, GlobalAveragePooling1D, Embedding
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
/opt/conda/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
df = pd.read_csv('./../input/train.csv')

**This is what the data looks like. It is a CSV file. 

**We have a column titled "text" containing sentences and a column titled "author" containing author initials.****

In [3]:
sample = df.values[0:3]
print(sample)
df.head(20)


[['id26305'
  'This process, however, afforded me no means of ascertaining the dimensions of my dungeon; as I might make its circuit, and return to the point whence I set out, without being aware of the fact; so perfectly uniform seemed the wall.'
  'EAP']
 ['id17569'
  'It never once occurred to me that the fumbling might be a mere mistake.'
  'HPL']
 ['id11008'
  'In his left hand was a gold snuff box, from which, as he capered down the hill, cutting all manner of fantastic steps, he took snuff incessantly with an air of the greatest possible self satisfaction.'
  'EAP']]


id                                               text author
0   id26305  This process, however, afforded me no means of...    EAP
1   id17569  It never once occurred to me that the fumbling...    HPL
2   id11008  In his left hand was a gold snuff box, from wh...    EAP
3   id27763  How lovely is spring As we looked from Windsor...    MWS
4   id12958  Finding nothing else, not even gold, the Super...    HPL
5   id22965  A youth passed in solitude, my best years spen...    MWS
6   id09674  The astronomer, perhaps, at this point, took r...    EAP
7   id13515        The surcingle hung in ribands from my body.    EAP
8   id19322  I knew that you could not say to yourself 'ste...    EAP
9   id00912  I confess that neither the structure of langua...    MWS
10  id16737  He shall find that I can feel my injuries; he ...    MWS
11  id16607  Here we barricaded ourselves, and, for the pre...    EAP
12  id19764  Herbert West needed fresh bodies because his l...    HPL
13  id18886  The farm like grounds extended back very deepl...    HPL
14  id17189   But a glance will show the fallacy of this idea.    EAP
15  id12799  He had escaped me, and I must commence a destr...    MWS
16  id08441  To these speeches they gave, of course, their ...    EAP
17  id13117  Her native sprightliness needed no undue excit...    MWS
18  id14862  I even went so far as to speak of a slightly h...    EAP
19  id20836  His facial aspect, too, was remarkable for its...    HPL

**Preprocess Text:**    
    1) Add a space on both sides of punctuation to avoid forming new words based off of punctuation.
    2) Convert author label to categorical label for better compatability with classification algorithms.

In [4]:
def preprocess(text):
    text = text.strip()
    text = text.replace("' ", " ' ")
    signs = set(',.:;"?!')
    prods = set(text) & signs
    if not prods:
        return text

    for sign in prods:
        text = text.replace(sign, ' {} '.format(sign) )
    return text

In [5]:
a2c = {'EAP': 0, 'HPL' : 1, 'MWS' : 2}
y = np.array([a2c[a] for a in df.author])
y = to_categorical(y)

**An N-gram is a contiguous sequence of n items from a given sequence of text or speech. The items can be phonemes, syllables, letters, words or base pairs according to the application (e.g. 2-gram sequence: "to be, be or, or not, not to, to be"; 3-gram sequence: "to be or, be or not, or not to, not to be".

**Next I visualize some N-grams for Edgar Allan Poe.****

In [6]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from plotly import tools
import plotly.plotly as py
from collections import Counter
from nltk.util import ngrams
from nltk.corpus import stopwords
import re


tokenize_regex = re.compile("[\w]+")
sw = set(stopwords.words("english"))

def preprocessText(text, ngram_order):
    """
    Transform text into a list of ngrams. Feel free to play with the order parameter
    """
    text = text.lower()
    
    text = [" ".join(ngram) for ngram in ngrams((tokenize_regex.findall(text)), ngram_order) \
            if (set(ngram) - sw)] # instead of filtering stopwords, let's just filter out the ngrams
                                  # with nothing but stopwords
    return text

def draw_word_histogram(texts, title, bars=30):
    """
    Draw a barplot for word frequency distribution.
    """
    # first, do the counting
    ngram_counter = Counter()
    for text in texts:
        ngram_counter.update(text)
    # for plotly, we need two lists: xaxis values and the corresponding yaxis values
    # this is how we split a list of two-element tuples into two lists
    features, counts = zip(*ngram_counter.most_common(bars))
    # now let's define the barplot
    bars = go.Bar(
        x=counts[::-1],  # inverse the values to have the largest on the top
        y=features[::-1],
        orientation="h",  # this makes it a horizontal barplot 
        marker=dict(
            color='rgb(128, 0, 32)'  # this color is called oxblood... spooky, isn't it?
        )
    )
    # this is how we customize the looks of our barplot
    layout = go.Layout(
        paper_bgcolor='rgb(0, 0, 0)',  # color of the background under the title and in the margins
        plot_bgcolor='rgb(0, 0, 0)',  # color of the plot background
        title=title,
        autosize=False,  # otherwise the plot would be too small to contain axis labels
        width=600,
        height=800,
        margin=go.Margin(
            l=120, # to make space for y-axis labels
        ),
        font=dict(
            family='Serif',
            size=13, # a lucky number
            color='rgb(200, 200, 200)'
        ),
        xaxis=dict(
            showgrid=True,  # all the possible lines - try switching them off
            zeroline=True,
            showline=True,
            zerolinecolor='rgb(200, 200, 200)',
            linecolor='rgb(200, 200, 200)',
            gridcolor='rgb(200, 200, 200)',
        ),
        yaxis=dict(
            ticklen=8  # to add some space between yaxis labels and the plot
        )
        
    )
    fig = go.Figure(data=[bars], layout=layout)
    iplot(fig, filename='h-bar')
    return


In [7]:
init_notebook_mode(connected=True)


poe = df[df.author=="EAP"].text.apply(preprocessText, ngram_order=1)
draw_word_histogram(poe, "Edgar Allan Poe Most Common Mono-grams")

In [8]:
poe = df[df.author=="EAP"].text.apply(preprocessText, ngram_order=2)
draw_word_histogram(poe, "Edgar Allan Poe Most Common Bi-grams")

In [9]:
poe = df[df.author=="EAP"].text.apply(preprocessText, ngram_order=3)
draw_word_histogram(poe, "Edgar Allan Poe Most Common Tri-grams")

In [10]:
poe = df[df.author=="EAP"].text.apply(preprocessText, ngram_order=4)
draw_word_histogram(poe, "Edgar Allan Poe Most Common Quad-grams")

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning:

generator 'ngrams' raised StopIteration



Here we create N-grams for N = (1 to 4) for each author.  These N-grams will be our features for classification.

In [11]:
def create_docs(df, n_gram_max=4):
    def add_ngram(q, n_gram_max):
            ngrams = []
            for n in range(1, n_gram_max+1):
                for w_index in range(len(q)-n+1):
                    ngrams.append('--'.join(q[w_index:w_index+n]))
            return q + ngrams
        
    docs = []
    for doc in df.text:
        doc = preprocess(doc).split()
        docs.append(' '.join(add_ngram(doc, n_gram_max)))
    
    return docs

In [12]:
min_count = 15

docs = create_docs(df)
tokenizer = Tokenizer(lower=True, filters='')
tokenizer.fit_on_texts(docs)
num_words = sum([1 for _, v in tokenizer.word_counts.items() if v >= min_count])

tokenizer = Tokenizer(num_words=num_words, lower=True, filters='')
tokenizer.fit_on_texts(docs)
docs = tokenizer.texts_to_sequences(docs)

maxlen = None

docs = pad_sequences(sequences=docs, maxlen=maxlen)

**Here I use Keras to build a custom neural network for this natural language processing problem.  

**Note that accuracy can be improved by increasing the number of epochs while speed can be improved by decreasing the number of epochs.****

In [13]:
input_dim = np.max(docs) + 1
embedding_dims = 20

In [14]:
def create_model(embedding_dims=20, optimizer='adam'):
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=embedding_dims))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(3, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
    return model

In [15]:
epochs = 20
x_train, x_test, y_train, y_test = train_test_split(docs, y, test_size=0.2)

model = create_model()
hist = model.fit(x_train, y_train,
                 batch_size=16,
                 validation_data=(x_test, y_test),
                 epochs=epochs,
                 callbacks=[EarlyStopping(patience=2, monitor='val_loss')])

Train on 15663 samples, validate on 3916 samples
Epoch 1/20
15663/15663 [==============================] - 9s - loss: 1.0861 - acc: 0.4022 - val_loss: 1.0815 - val_acc: 0.4091
Epoch 2/20
15663/15663 [==============================] - 10s - loss: 1.0747 - acc: 0.4038 - val_loss: 1.0621 - val_acc: 0.4104
Epoch 3/20
15663/15663 [==============================] - 9s - loss: 1.0433 - acc: 0.4396 - val_loss: 1.0216 - val_acc: 0.4803
Epoch 4/20
15663/15663 [==============================] - 9s - loss: 0.9873 - acc: 0.5386 - val_loss: 0.9592 - val_acc: 0.5718
Epoch 5/20
15663/15663 [==============================] - 9s - loss: 0.9164 - acc: 0.6425 - val_loss: 0.8934 - val_acc: 0.6141
Epoch 6/20
15663/15663 [==============================] - 9s - loss: 0.8441 - acc: 0.7016 - val_loss: 0.8283 - val_acc: 0.6971
Epoch 7/20
15663/15663 [==============================] - 9s - loss: 0.7785 - acc: 0.7372 - val_loss: 0.7758 - val_acc: 0.7439
Epoch 8/20
15663/15663 [==============================] - 9s 

**>80% Accuracy!  Great!  Next, we can improve the accuracy by increasing the number of epochs.**

In [16]:
# test_df = pd.read_csv('../input/test.csv')
# docs = create_docs(test_df)
# docs = tokenizer.texts_to_sequences(docs)
# docs = pad_sequences(sequences=docs, maxlen=maxlen)
# y = model.predict_proba(docs)

# result = pd.read_csv('../input/sample_submission.csv')
# for a, i in a2c.items():
#     result[a] = y[:, i]
# result.to_csv('_new_submission_1.csv', index=False)
